<a href="https://colab.research.google.com/github/TamandaCodes/BrodgarFNO/blob/main/Transient_Model_Pre_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 01 Brodgar Transient Model Pre-Processing Notebook

In [ ]:
!pip install pypontem

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 7.1 MB/s eta 0:00:00


In [ ]:
# ================================
#  EXTRACT ALL PPL CASES TO CSV
# ================================

import os, sys, tempfile, time
import pandas as pd
from google.colab import drive
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from pypontem.ppl.pplparser import pplParser, pplBatchParser

# ---------------------------------------------------------------------
# Mount & project setup
# ---------------------------------------------------------------------
drive_name = '/content/gdrive'
if not os.path.ismount(drive_name):
    drive.mount(drive_name)

dir_path = '/content/gdrive/MyDrive/Colab Notebooks/Transient Folder'
os.makedirs(dir_path, exist_ok=True)
sys.path.insert(1, dir_path)

try:
    DIR = dir_path  # type: ignore[name-defined]
except Exception:
    DIR = ""

def P(fn: str) -> str:
    return os.path.join(DIR, fn)




Mounted at /content/gdrive


In [ ]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

SERVICE_ACCOUNT_FILE = P('client_secret_1010711658291-u1vvb0jpj1euh013u6ubppd3cmagg8vc.apps.googleusercontent.com.json')

# Google Drive access scope
SCOPES = ['https://www.googleapis.com/auth/drive']

flow = InstalledAppFlow.from_client_secrets_file(
    SERVICE_ACCOUNT_FILE,
    scopes=SCOPES,
    redirect_uri='urn:ietf:wg:oauth:2.0:oob'  # special URI for installed apps / console-based
)

# Generate URL
auth_url, _ = flow.authorization_url(
    access_type='offline',
    prompt='consent select_account',
    include_granted_scopes='true'
)

# Show the URL to the user
print(f"🔗 Please go to this URL and choose the account to authorize:\n\n{auth_url}")

# Get the authorization code from the user
code = input("🔐 Paste the authorization code here: ")

# Exchange the code for credentials
flow.fetch_token(code=code)
creds = flow.credentials

# Build the Google Drive service
drive_service = build('drive', 'v3', credentials=creds)

🔗 Please go to this URL and choose the account to authorize:

https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1010711658291-u1vvb0jpj1euh013u6ubppd3cmagg8vc.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=vl1rfke1F2gwHQfF2oh6MFqa4kk846&access_type=offline&prompt=consent+select_account&include_granted_scopes=true
🔐 Paste the authorization code here: 4/1Ab32j92DG5WIhFdH-o-vSdh1rghDzobE1c6q7ruQwpVzP73rY0p8J38NTJs


In [ ]:
# ---------------------------------------------------------------------
# Locate all PPL files in target Drive folder
# ---------------------------------------------------------------------
FOLDER_ID = '1eQAiSsTQX7SUI2Anx9D-MCwU3pQ-zdaJ'
files = []
page_token = None

print("🔍 Searching for PPL files in Drive folder...")
while True:
    results = drive_service.files().list(
        q=f"'{FOLDER_ID}' in parents and mimeType!='application/vnd.google-apps.folder' and name contains '.ppl' ",
        fields="files(id, name)",
        supportsAllDrives=True,
        includeItemsFromAllDrives=True,
        pageSize=1000,
        pageToken=page_token
    ).execute()

    files.extend(results.get('files', []))
    page_token = results.get('nextPageToken', None)
    if page_token is None:
        break

ppl_files_meta = [f for f in files if f['name'].endswith('.ppl')]
print(f"✅ Found {len(ppl_files_meta)} .ppl files in folder")

🔍 Searching for PPL files in Drive folder...
✅ Found 150 .ppl files in folder


In [ ]:
# ---------------------------------------------------------------------
# Define the extraction matrix
# ---------------------------------------------------------------------

matrix = {
    "Variable": ["PT", "TM", "HOL", "HOLWT", "UG","UL","ULWT"], # Updated variables based on partial catalog
    "BRANCH":  ["BRANCH_FLOWLINE"] * 7,
    "Units":   ["bara", "degC", "", "", "m_s", "m_s", "m_s"], # Updated units based on partial catalog and common velocity units
    "Profile_units": ["m"] * 7,
    "Time_units": ["min"] * 7,
    "Start_time": None,
    "End_time": None
}


## Check properties of the .ppl files

In [ ]:
# ---------------------------------------------------------------------
# Sequential extraction - Attempting to inspect one file
# ---------------------------------------------------------------------
total = len(ppl_files_meta)
print(f"\n🚀 Attempting to inspect the first PPL file...")

# Process only the first file for inspection
file_meta = ppl_files_meta[0]
file_name = file_meta["name"]
file_id   = file_meta["id"]
case_name = os.path.splitext(file_name)[0]
csv_path  = P(f"{case_name}.csv") # Still define for consistency, but won't save

print(f"[1/{total}] ⬇️  Downloading {file_name} ...")
temp_path = os.path.join(tempfile.gettempdir(), file_name)

# --- download with retries ---
MAX_RETRIES = 3 # Define MAX_RETRIES locally for this cell
for attempt in range(1, MAX_RETRIES + 1):
    try:
        with open(temp_path, "wb") as temp_file:
            request = drive_service.files().get_media(fileId=file_id)
            downloader = MediaIoBaseDownload(temp_file, request)
            done = False
            while not done:
                status, done = downloader.next_chunk()
                if status:
                    print(f"   Download {int(status.progress() * 100)}%", end="\r")
        print(f"\n   Download complete.")
        break  # success
    except Exception as e:
        if attempt == MAX_RETRIES:
            print(f"\n   ❌ Download failed after {MAX_RETRIES} attempts: {e}")
            if os.path.exists(temp_path):
                os.remove(temp_path)
            # Exit this cell's execution as download failed
            raise e # Re-raise the exception to stop execution if download fails
        print(f"\n   Download failed (attempt {attempt}/{MAX_RETRIES}): {e}. Retrying...")
        time.sleep(1.5 * attempt)  # backoff


# Inspect the .ppl file
try:
    print(f"\n   🔍 Inspecting {file_name}...")
    # Use pplParser to inspect the file structure
    ppl_obj = pplParser(temp_path)

    print("\n   --- PPL File Structure (Partial) ---")

    # Access and print relevant attributes
    if hasattr(ppl_obj, 'branch_names'):
        print(f"   Branch Names: {ppl_obj.branch_names}")
    if hasattr(ppl_obj, 'branch_profiles'):
        print(f"   Branch Profiles (keys): {ppl_obj.branch_profiles.keys()}") # Print keys as profiles can be large
    if hasattr(ppl_obj, 'catalog'):
        print(f"   Catalog ( items): {list(ppl_obj.catalog.items())}...") # Print first few catalog items
    if hasattr(ppl_obj, 'search_catalog'):
         print(f"   Search Catalog (first 20 items): {list(ppl_obj.search_catalog.items())[:20]}...") # Print first few search catalog items

    # As a fallback, still show attributes
    print(f"   pplParser object attributes: {dir(ppl_obj)}")


except Exception as e:
    print(f"   ❌ Error inspecting {file_name}: {e}")
finally:
    # Clean up the temporary file
    if os.path.exists(temp_path):
        os.remove(temp_path)


print("\n✅ Inspection attempt complete. Please examine the output above to understand the file structure.")


🚀 Attempting to inspect the first PPL file...
[1/150] ⬇️  Downloading 142_TR_MainPipelineOnly_TunedPre2021Pressures_CGR50_WC60_Q78_TIN52_AP29.ppl ...
   Download 100%
   Download complete.

   🔍 Inspecting 142_TR_MainPipelineOnly_TunedPre2021Pressures_CGR50_WC60_Q78_TIN52_AP29.ppl...

   --- PPL File Structure (Partial) ---
   Branch Names: ["'BRANCH_FLOWLINE'"]
   Branch Profiles (keys): dict_keys(['BRANCH_FLOWLINE'])
   Catalog ( items): [('Variable', 0        ID
1        QG
2      QGST
3      QLST
4       QLT
5     QLTHL
6     QLTWT
7      QOST
8      QWST
9        UG
10       UL
11     ULWT
12    DTHYD
13      HOL
14    HOLHL
15    HOLWT
16       PT
17       TM
18       TU
19      TWS
20       WC
Name: Variable, dtype: object), ('Locator Type', 0     BOUNDARY:
1     BOUNDARY:
2     BOUNDARY:
3     BOUNDARY:
4     BOUNDARY:
5     BOUNDARY:
6     BOUNDARY:
7     BOUNDARY:
8     BOUNDARY:
9     BOUNDARY:
10    BOUNDARY:
11    BOUNDARY:
12     SECTION:
13     SECTION:
14     SECTION:


## Parsing to CSV files


In [ ]:
# ---------------------------------------------------------------------
# Parallel extraction
# ---------------------------------------------------------------------
import concurrent.futures as cf
from concurrent.futures import as_completed
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import tempfile
import pandas as pd
import os, time, io, traceback

# Limit to first 200 cases if needed
ppl_files_meta = ppl_files_meta[:150]

MAX_WORKERS = 8           # tune this (4–8 is a good start)
MAX_RETRIES = 3           # retry count for download/parse

def download_and_parse_one(file_meta):
    """
    Worker: downloads one .ppl, parses it, writes one CSV.
    Returns (file_name, csv_path, ok:bool, message:str)
    """
    file_name = file_meta["name"]
    file_id   = file_meta["id"]
    case_name = os.path.splitext(file_name)[0]
    csv_path  = P(f"{case_name}.csv")

    if os.path.exists(csv_path):
        return (file_name, csv_path, True, "already_exists")

    # Build a Drive service in this thread (safer than sharing)
    service = build('drive', 'v3', credentials=creds)

    # Unique temp path for each worker
    tmp = tempfile.NamedTemporaryFile(prefix=case_name + "_", suffix=".ppl", delete=False)
    tmp_path = tmp.name
    tmp.close()

    # --- download with retries ---
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            with open(tmp_path, "wb") as f:
                request = service.files().get_media(fileId=file_id)
                downloader = MediaIoBaseDownload(f, request)
                done = False
                while not done:
                    status, done = downloader.next_chunk()
            break  # success
        except Exception as e:
            if attempt == MAX_RETRIES:
                try:
                    os.remove(tmp_path)
                except Exception:
                    pass
                return (file_name, csv_path, False, f"download_failed: {e}")
            time.sleep(1.5 * attempt)  # backoff

    # --- parse with retries ---
    try:
        from pypontem.ppl.pplparser import pplParser, pplBatchParser
    except Exception as e:
        try:
            os.remove(tmp_path)
        except Exception:
            pass
        return (file_name, csv_path, False, f"import_failed: {e}")

    for attempt in range(1, MAX_RETRIES + 1):
        try:
            # Use batch API even for a single file (consistent schema)
            batch   = pplBatchParser([tmp_path])
            profiles = batch.extract_profiles(matrix)
            profiles.columns = profiles.columns.droplevel(0)

            # write CSV
            profiles.to_csv(csv_path, index=False)
            try:
                os.remove(tmp_path)
            except Exception:
                pass
            return (file_name, csv_path, True, "ok")
        except Exception as e:
            if attempt == MAX_RETRIES:
                try:
                    os.remove(tmp_path)
                except Exception:
                    pass
                return (file_name, csv_path, False, f"parse_failed: {e}")
            time.sleep(1.5 * attempt)  # backoff

# Kick off the pool
total = len(ppl_files_meta)
print(f"\n🚀 Starting PARALLEL extraction for {total} cases "
      f"(workers={MAX_WORKERS}, retries={MAX_RETRIES})...")

ok_count = 0
skip_count = 0
failures = []

with cf.ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
    future_map = {ex.submit(download_and_parse_one, meta): meta for meta in ppl_files_meta}

    for i, fut in enumerate(as_completed(future_map), start=1):
        try:
            file_name, csv_path, ok, msg = fut.result()
            if msg == "already_exists":
                skip_count += 1
                print(f"[{i}/{total}] ⏩ {file_name}: already exists -> {csv_path}")
            elif ok:
                ok_count += 1
                print(f"[{i}/{total}] ✅ {file_name} -> {csv_path}")
            else:
                failures.append((file_name, msg))
                print(f"[{i}/{total}] ❌ {file_name}: {msg}")
        except Exception as e:
            meta = future_map[fut]
            failures.append((meta["name"], f"worker_exception: {e}"))
            print(f"[{i}/{total}] ❌ {meta['name']}: worker_exception {e}")

print("\n✅ Parallel extraction complete!")
print(f"   Saved: {ok_count} | Skipped: {skip_count} | Failed: {len(failures)}")
if failures:
    print("   Failures:")
    for name, reason in failures[:20]:
        print(f"     - {name}: {reason}")
    if len(failures) > 20:
        print(f"     ... and {len(failures)-20} more")



🚀 Starting PARALLEL extraction for 150 cases (workers=6, retries=3)...
[1/150] ⏩ 142_TR_MainPipelineOnly_TunedPre2021Pressures_CGR50_WC60_Q78_TIN52_AP29.ppl: already exists -> /content/gdrive/MyDrive/Colab Notebooks/Transient Folder/142_TR_MainPipelineOnly_TunedPre2021Pressures_CGR50_WC60_Q78_TIN52_AP29.csv
[2/150] ✅ 149_TR_MainPipelineOnly_TunedPre2021Pressures_CGR50_WC60_Q94_TIN69_AP30.ppl -> /content/gdrive/MyDrive/Colab Notebooks/Transient Folder/149_TR_MainPipelineOnly_TunedPre2021Pressures_CGR50_WC60_Q94_TIN69_AP30.csv
[3/150] ✅ 150_TR_MainPipelineOnly_TunedPre2021Pressures_CGR50_WC60_Q113_TIN61_AP29.ppl -> /content/gdrive/MyDrive/Colab Notebooks/Transient Folder/150_TR_MainPipelineOnly_TunedPre2021Pressures_CGR50_WC60_Q113_TIN61_AP29.csv
[4/150] ✅ 147_TR_MainPipelineOnly_TunedPre2021Pressures_CGR50_WC60_Q52_TIN67_AP30.ppl -> /content/gdrive/MyDrive/Colab Notebooks/Transient Folder/147_TR_MainPipelineOnly_TunedPre2021Pressures_CGR50_WC60_Q52_TIN67_AP30.csv
[5/150] ✅ 146_TR_MainP

## Creating Tensor

In [ ]:
# build_tensor_from_case_csvs_ckpt20.py
import os, re, time, glob, numpy as np, pandas as pd, torch
from google.colab import drive # Import drive

# =========================
# Root dir & path helpers
# =========================
# Ensure dir_path is defined in this cell's scope
drive_name = '/content/gdrive'
if not os.path.ismount(drive_name):
    drive.mount(drive_name)

dir_path = '/content/gdrive/MyDrive/Colab Notebooks/Transient Folder'
os.makedirs(dir_path, exist_ok=True)
# sys.path.insert(1, dir_path) # This line might not be needed here and can cause issues

DIR = dir_path # Now DIR is definitely set to dir_path

def P(fn: str) -> str:
    # Use os.path.join directly with DIR
    return os.path.join(DIR, fn)

def find_input(filename: str) -> str:
    cands = [os.path.join(DIR, filename), os.path.join('/mnt/data', filename), filename]
    for p in cands:
        if os.path.exists(p):
            return p
    raise FileNotFoundError(f"Could not find input file '{filename}' in {cands}")

# =========================
# Config
# =========================
VARIABLES = ["PT", "TM", "HOL", "HOLWT", "UG","UL","ULWT"]   # channel order
N_TIMES = 288                                      # 0..120
TIME_MIN, TIME_MAX = 0, 287
CSV_GLOB = os.path.join(dir_path, "*.csv")        # Modified to use dir_path directly
CKPT_PATH  = P("full_dataset_building.pt")          # rolling checkpoint
FINAL_PATH = P("transient_model_full_dataset.pt")                   # final artifact
CHECKPOINT_EVERY = 50                              # ✅ checkpoint cadence

# =========================
# Parsing helpers
# =========================
_time_re = re.compile(r'_time_in_min:?_?(-?\d+(?:\.\d+)?)', re.IGNORECASE)
_var_re  = re.compile(r'_(PT|TM|HOL|HOLWT|UG|UL|ULWT)_', re.IGNORECASE) # Added UG, UL, ULWT to regex

def parse_var_and_time(colname: str):
    m_var = _var_re.search(colname)
    m_t   = _time_re.search(colname)
    if not (m_var and m_t):
        return None, None, None, False
    var = m_var.group(1).upper()
    try:
        t_minutes = float(m_t.group(1))  # e.g., 1070.012 minutes
    except Exception:
        return None, None, None, False

    # Convert minutes to 5-minute interval index
    # 1070.012 minutes / 5 = 214.0024 → rounds to index 214
    t_int = int(round(t_minutes / 5))

    # Clamp to valid range [0, 287] for 24 hours
    # (24 hours * 60 min / 5 = 288 intervals, indices 0-287)
    t_int = max(0, min(287, t_int))

    return var, t_int, t_minutes, True

def load_case_csv(path: str):
    df = pd.read_csv(path)
    if df.shape[1] < 3:
        raise ValueError(f"CSV has too few columns: {path}")
    sections = pd.to_numeric(df.iloc[:, 0], errors='coerce').to_numpy()
    print (sections[:10])  #i need to see if its having errors

    if np.isnan(sections).all():
        raise ValueError(f"First column is not numeric sections (m): {path}")

    mapping = {v: {} for v in VARIABLES}  # var -> {t_int: (col_idx, abs_delta)}
    for j in range(1, df.shape[1]):
        col = str(df.columns[j])
        var, t_int, t_float, ok = parse_var_and_time(col)
        if not ok or var not in mapping:
            continue
        delta = abs(t_float - round(t_int)) # Compare to rounded integer time
        prev = mapping[var].get(t_int)
        if (prev is None) or (delta < prev[1]):
            mapping[var][t_int] = (j, delta)
    return sections, mapping, df

def save_checkpoint(state: dict):
    os.makedirs(os.path.dirname(CKPT_PATH) or ".", exist_ok=True)
    torch.save(state, CKPT_PATH)
    print(f"💾 Checkpoint saved: {CKPT_PATH}")

def load_checkpoint():
    if os.path.exists(CKPT_PATH):
        try:
            return torch.load(CKPT_PATH, weights_only=False)
        except Exception:
            pass
    return None

# Global flag to control printing mismatch details only once
_mismatch_details_printed = False




In [ ]:
# =========================
# Main
# =========================
def main_build_from_csvs():
    global _mismatch_details_printed # Declare intent to use the global flag
    print("="*80)
    print("BUILD FULL DATASET TENSOR FROM PER-CASE CSVs (checkpoint every 50)")
    print("="*80)

    csv_paths = sorted(
        p for p in glob.glob(CSV_GLOB)
        if p.lower().endswith(".csv")
        and "comparison_errors" not in os.path.basename(p).lower()
        and "tm_outliers" not in os.path.basename(p).lower()
    )
    if not csv_paths:
        raise RuntimeError(f"No CSV files found with pattern: {CSV_GLOB}")

    # resume or initialize
    ck = load_checkpoint()
    if ck is not None:
        print("↩️  Resuming from checkpoint.")
        full_tensor   = ck["full_tensor"]                 # [C, X, 7, 288]
        case_filled   = ck["case_filled"]                 # [C] bool
        ref_sections  = ck["section_lengths"].numpy()
        time_hours    = ck["time_hours"].numpy()
        case_names    = ck["case_names"]
        start_idx     = int(ck.get("next_case_idx", 0))
        # Reset the print flag if resuming from a checkpoint
        _mismatch_details_printed = False
    else:
        # bootstrap using first CSV
        print(f"✨ Initializing with sections from: {os.path.basename(csv_paths[0])}")
        try:
            ref_sections, _, _ = load_case_csv(csv_paths[0])
        except Exception as e:
             raise RuntimeError(f"Failed to load or parse the first CSV ({os.path.basename(csv_paths[0])}) to get reference sections: {e}") from e

        C = len(csv_paths)
        X = len(ref_sections)
        print(X)

        full_tensor = torch.full((C, X, len(VARIABLES), N_TIMES), float('nan'), dtype=torch.float32)
        case_filled = torch.zeros(C, dtype=torch.bool)
        time_hours  = np.arange(N_TIMES, dtype=np.float32)
        case_names  = [os.path.splitext(os.path.basename(p))[0] for p in csv_paths]
        start_idx   = 0
        save_checkpoint({
            "full_tensor": full_tensor,
            "case_filled": case_filled,
            "section_lengths": torch.tensor(ref_sections, dtype=torch.float32),
            "time_hours": torch.tensor(time_hours, dtype=torch.float32),
            "variable_names": VARIABLES,
            "case_names": case_names,
            "next_case_idx": start_idx
        })
        _mismatch_details_printed = False


    total = len(csv_paths)
    print(f"📄 Found {total} CSVs. Starting at index {start_idx}.")
    print(f"Channels: {VARIABLES} | Time grid: 0..{N_TIMES-1}min")
    last_ckpt_i = start_idx

    for i in range(start_idx, total):
        fname = os.path.basename(csv_paths[i])
        if case_filled[i]:
            print(f"[{i+1}/{total}] ⏩ Already filled: {fname}")
            continue

        try:
            sections, mapping, df = load_case_csv(csv_paths[i])

            # quick section sanity (must match reference exactly for speed)
            # Relaxed tolerance for np.allclose
            if len(sections) != len(ref_sections) or not np.allclose(sections, ref_sections, rtol=0, atol=1e-3, equal_nan=True): # Increased atol
                print(f"[{i+1}/{total}] ⚠️ Skipping {fname}: Sections mismatch ({len(sections)} vs {len(ref_sections)} or values differ).")

                # Print mismatch details only for the first time it occurs
                if not _mismatch_details_printed:
                    print("   --- Section Mismatch Details (First occurrence) ---")
                    print(f"   File: {fname}")
                    print(f"   Reference Sections (first 20): {ref_sections[:20]}")
                    print(f"   Current File Sections (first 20): {sections[:20]}")
                    # Find indices where values differ beyond tolerance
                    diff_indices = np.where(~np.isclose(sections, ref_sections, rtol=0, atol=1e-3))[0]
                    print(f"   Indices with significant difference (first 10): {diff_indices[:10]}...")
                    if len(diff_indices) > 0:
                         print(f"   Example at first diff index ({diff_indices[0]}): Ref={ref_sections[diff_indices[0]]}, Current={sections[diff_indices[0]]}")
                    print("   --------------------------------------------------")
                    _mismatch_details_printed = True # Set the flag

                # We don't mark case_filled as True, so it will be attempted again if tolerance is changed or files are fixed.
                # We also don't save a checkpoint on this specific type of non-fatal error.
                continue # Skip to the next file

            X = len(ref_sections)
            case_arr = np.full((X, len(VARIABLES), N_TIMES), np.nan, dtype=np.float32)

            for v_idx, var in enumerate(VARIABLES):
                for t_int, (col_idx, _) in mapping[var].items():
                    vals = pd.to_numeric(df.iloc[:, col_idx], errors='coerce').to_numpy(dtype=np.float32)
                    if len(vals) != X:
                        raise ValueError(f"Length mismatch in {fname}, var {var}, time {t_int}: {len(vals)} vs {X}")
                    case_arr[:, v_idx, t_int] = vals

            full_tensor[i] = torch.from_numpy(case_arr)
            case_filled[i] = True
            print(f"[{i+1}/{total}] ✅ {fname}")

        except Exception as e:
            print(f"[{i+1}/{total}] ❌ {fname}: {e}")
            # checkpoint immediately on other errors so you can resume past it
            save_checkpoint({
                "full_tensor": full_tensor,
                "case_filled": case_filled,
                "section_lengths": torch.tensor(ref_sections, dtype=torch.float32),
                "time_hours": torch.tensor(time_hours, dtype=torch.float32),
                "variable_names": VARIABLES,
                "case_names": case_names,
                "next_case_idx": i+1
            })
            last_ckpt_i = i+1
            continue

        # periodic checkpoint (every CHECKPOINT_EVERY processed)
        if (i + 1 - start_idx) > 0 and (i + 1 - start_idx) % CHECKPOINT_EVERY == 0: # Adjusted checkpoint condition
             save_checkpoint({
                "full_tensor": full_tensor,
                "case_filled": case_filled,
                "section_lengths": torch.tensor(ref_sections, dtype=torch.float32),
                "time_hours": torch.tensor(time_hours, dtype=torch.float32),
                "variable_names": VARIABLES,
                "case_names": case_names,
                "next_case_idx": i+1
            })
             last_ckpt_i = i+1


    # final checkpoint if the last block < CHECKPOINT_EVERY or all processed
    if last_ckpt_i != total or start_idx == 0: # Ensure final checkpoint is saved
         save_checkpoint({
            "full_tensor": full_tensor,
            "case_filled": case_filled,
            "section_lengths": torch.tensor(ref_sections, dtype=torch.float32),
            "time_hours": torch.tensor(time_hours, dtype=torch.float32),
            "variable_names": VARIABLES,
            "case_names": case_names,
            "next_case_idx": total # Indicate all cases attempted
        })


    # Save final dataset - only if all expected cases were filled
    if case_filled.all():
        final_data = {
            "full_tensor": full_tensor,
            "section_lengths": torch.tensor(ref_sections, dtype=torch.float32),
            "time_hours": torch.tensor(time_hours, dtype=torch.float32),
            "variable_names": VARIABLES,
            "case_numbers": [i+1 for i, filled in enumerate(case_filled) if filled], # Only include case numbers for filled cases
            "case_names_filled": [case_names[i] for i, filled in enumerate(case_filled) if filled], # Add names for filled cases
            "metadata": {
                "creation_date": time.strftime('%Y-%m-%d %H:%M:%S'),
                "source": "Per-case CSVs parsed from PPL profiles",
                "notes": f"Times rounded to nearest hour; checkpointed every {CHECKPOINT_EVERY} cases. Skipped cases with section mismatches."
            }
        }
        torch.save(final_data, FINAL_PATH)
        print(f"\n🎉 Done. Saved final dataset: {FINAL_PATH}")
        print(f"Tensor shape: {tuple(full_tensor.shape)} (cases, sections, variables, time)")
        print(f"Cases filled: {int(case_filled.sum().item())}/{total}")
    else:
        print(f"\n⚠️ Not all cases were filled ({int(case_filled.sum().item())}/{total}). Final dataset not saved.")
        print("Skipped cases likely had section mismatches or other errors.")


if __name__ == "__main__":
    main_build_from_csvs()

BUILD FULL DATASET TENSOR FROM PER-CASE CSVs (checkpoint every 50)
✨ Initializing with sections from: 100_TR_MainPipelineOnly_TunedPre2021Pressures_CGR50_WC30_Q113_TIN61_AP29.csv
[  5.     20.     41.     66.5    95.5   122.507 147.521 172.528 197.529
 223.029]
1279
💾 Checkpoint saved: /content/gdrive/MyDrive/Colab Notebooks/Transient Folder/full_dataset_building.pt
📄 Found 150 CSVs. Starting at index 0.
Channels: ['PT', 'TM', 'HOL', 'HOLWT', 'UG', 'UL', 'ULWT'] | Time grid: 0..287min
[  5.     20.     41.     66.5    95.5   122.507 147.521 172.528 197.529
 223.029]
[1/150] ✅ 100_TR_MainPipelineOnly_TunedPre2021Pressures_CGR50_WC30_Q113_TIN61_AP29.csv
[  5.     20.     41.     66.5    95.5   122.507 147.521 172.528 197.529
 223.029]
[2/150] ✅ 101_TR_MainPipelineOnly_TunedPre2021Pressures_CGR50_WC60_Q106_TIN65_AP29.csv
[  5.     20.     41.     66.5    95.5   122.507 147.521 172.528 197.529
 223.029]
[3/150] ✅ 102_TR_MainPipelineOnly_TunedPre2021Pressures_CGR50_WC60_Q46_TIN60_AP29.csv
[

In [ ]:
import torch
import os


if 'FINAL_PATH' in locals() or 'FINAL_PATH' in globals():
    if os.path.exists(FINAL_PATH):
        print(f"Loading tensor file: {FINAL_PATH}")
        try:
            loaded_data = torch.load(FINAL_PATH)

            print("\n--- Tensor File Contents ---")
            # Check if the loaded data is a dictionary (as saved by the previous script)
            if isinstance(loaded_data, dict):
                print("Loaded data is a dictionary. Keys available:")
                for key in loaded_data.keys():
                    print(f"- {key}")

                # Assuming the main tensor is stored under the key "full_tensor"
                if "full_tensor" in loaded_data:
                    full_tensor = loaded_data["full_tensor"]
                    print(f"\nShape of the main tensor ('full_tensor'): {tuple(full_tensor.shape)}")
                    print(f"Dimensions: (cases, sections, variables, time)")
                    print(f"Data type: {full_tensor.dtype}")

                # You can also inspect other parts of the dictionary
                if "variable_names" in loaded_data:
                     print(f"\nVariable names: {loaded_data['variable_names']}")
                if "section_lengths" in loaded_data:
                     print(f"Shape of section lengths: {loaded_data['section_lengths'].shape}")
                     print(f"First 10 section lengths: {loaded_data['section_lengths'][:10].tolist()}")
                if "time_hours" in loaded_data:
                     print(f"Shape of time hours: {loaded_data['time_hours'].shape}")
                     print(f"First 10 time hours: {loaded_data['time_hours'][:10].tolist()}")


            elif isinstance(loaded_data, torch.Tensor):
                print("Loaded data is a raw tensor.")
                print(f"Shape of the tensor: {tuple(loaded_data.shape)}")
                print(f"Data type: {loaded_data.dtype}")
            else:
                print(f"Loaded data is of unknown type: {type(loaded_data)}")


        except Exception as e:
            print(f"Error loading or inspecting tensor file: {e}")
            print("Please ensure the previous cell successfully saved the tensor file.")

    else:
        print(f"Tensor file not found at: {FINAL_PATH}")
        print("Please ensure the previous cell has been run successfully to generate the file.")
else:
    print("FINAL_PATH variable is not defined.")
    print("Please ensure the previous cell defining FINAL_PATH has been run.")

FINAL_PATH variable is not defined.
Please ensure the previous cell defining FINAL_PATH has been run.


## Concatenate the two tensors


In [ ]:
import os, re, time, glob, numpy as np, pandas as pd, torch
from google.colab import drive # Import drive

# =========================
# Root dir & path helpers
# =========================
# Ensure dir_path is defined in this cell's scope
drive_name = '/content/gdrive'
if not os.path.ismount(drive_name):
    drive.mount(drive_name)

# Change dir_path back to the user's personal drive where directory creation is supported
dir_path = '/content/gdrive/MyDrive/Colab Notebooks/Transient Folder'
os.makedirs(dir_path, exist_ok=True)
# sys.path.insert(1, dir_path) # This line might not be needed here and can cause issues

DIR = dir_path # Now DIR is definitely set to dir_path

def P(fn: str) -> str:
    # Use os.path.join directly with DIR
    return os.path.join(DIR, fn)

def find_input(filename: str) -> str:
    cands = [os.path.join(DIR, filename), os.path.join('/mnt/data', filename), filename]
    for p in cands:
        if os.path.exists(p):
            return p
    raise FileNotFoundError(f"Could not find input file '{filename}' in {cands}")

Mounted at /content/gdrive


In [ ]:
# Assuming dir_path is defined from the initial setup cells
import os
import torch

# Define the path to the FNO_dataset.pt file
try:
    DIR = dir_path # Ensure DIR is set from dir_path
except NameError:
    print("Error: dir_path is not defined. Please run the initial setup cells.")
    raise # Stop execution if dir_path is not available

def P(fn: str) -> str:
    return os.path.join(DIR, fn)

fno_dataset_path = P("transient_model_full_dataset.pt")


# Load the FNO_dataset.pt file
try:
    loaded_data = torch.load(fno_dataset_path)
    print(f"Successfully loaded data from: {fno_dataset_path}")

    # You can add code here to inspect the loaded_data if needed
    if isinstance(loaded_data, dict):
        print("Loaded data is a dictionary. Keys:")
        for key in loaded_data.keys():
            print(f"- {key}")
        # Example: if the tensor is under the key 'full_tensor'
        if 'full_tensor' in loaded_data:
            print(f"Shape of 'full_tensor': {loaded_data['full_tensor'].shape}")
    elif isinstance(loaded_data, torch.Tensor):
        print(f"Loaded data is a tensor with shape: {loaded_data.shape}")
    else:
        print(f"Loaded data is of type: {type(loaded_data)}")


except FileNotFoundError:
    print(f"Error: File not found at {fno_dataset_path}")
    print("Please ensure the 'FNO_dataset.pt' file exists at this location.")
except Exception as e:
    print(f"An error occurred while loading the file: {e}")

Successfully loaded data from: /content/gdrive/MyDrive/Colab Notebooks/Transient Folder/transient_model_full_dataset.pt
Loaded data is a dictionary. Keys:
- full_tensor
- section_lengths
- time_hours
- variable_names
- case_numbers
- case_names_filled
- metadata
Shape of 'full_tensor': torch.Size([150, 1279, 7, 288])


In [ ]:
# Display the loaded_data dictionary
display(loaded_data)

{'full_tensor': tensor([[[[8.9043e+01, 8.8747e+01, 8.8669e+01,  ..., 8.5078e+01,
            8.4863e+01, 8.4476e+01],
           [6.0928e+01, 5.9900e+01, 5.9656e+01,  ..., 5.2493e+01,
            5.1918e+01, 5.1884e+01],
           [1.1600e-01, 1.1700e-01, 1.1700e-01,  ..., 1.2200e-01,
            1.2200e-01, 1.2200e-01],
           ...,
           [6.5480e+01, 6.5480e+01, 6.5480e+01,  ..., 6.5480e+01,
            6.5480e+01, 6.5480e+01],
           [2.8880e+01, 2.8880e+01, 2.8880e+01,  ..., 2.8880e+01,
            2.8880e+01, 2.8880e+01],
           [1.0586e+02, 1.0586e+02, 1.0586e+02,  ..., 1.0586e+02,
            1.0586e+02, 1.0586e+02]],
 
          [[8.8994e+01, 8.8699e+01, 8.8620e+01,  ..., 8.5037e+01,
            8.4820e+01, 8.4434e+01],
           [6.0784e+01, 5.9805e+01, 5.9529e+01,  ..., 5.2339e+01,
            5.1815e+01, 5.1753e+01],
           [1.0900e-01, 1.1000e-01, 1.1000e-01,  ..., 1.1400e-01,
            1.1400e-01, 1.1400e-01],
           ...,
           [6.5480e+01,

In [ ]:
c = torch.split(loaded_data['full_tensor'], 50, dim=0)
batch_50 = c[0]
print(batch_50.shape)

torch.Size([50, 1279, 7, 288])


In [ ]:
import pandas as pd
import numpy as np
import torch

# Load csv without index column
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Transient Folder/FNO Global variables2.csv') # Removed index_col=0
print(df.shape)
print(df.head())

# Convert to array - assuming the first column is the case number and the rest are features
# Drop rows with all NaN values before converting to numpy
df_cleaned = df.dropna(how='all')
features = df[['TT','PI','FI']].values #the three global variables
tensor = features.reshape(50, 289, 3)
extra_tensor = tensor[:,:-1,:]

print(extra_tensor.shape)



(14450, 5)
   Path        Date-Time     TT     PI      FI
0     1  01/01/2025 0:00  65.48  28.88  105.86
1     1  01/01/2025 0:05  64.12  28.96  103.92
2     1  01/01/2025 0:10  63.91  28.96  103.70
3     1  01/01/2025 0:15  64.59  28.91  105.58
4     1  01/01/2025 0:20  65.28  28.88  105.33
(50, 288, 3)


In [ ]:

# Convert to tensor first
extra_tensor = torch.from_numpy(extra_tensor).float()  # [50, 288,3]

# Reshape and expand
extra_tensor = extra_tensor.unsqueeze(1)
extra_tensor = extra_tensor.permute(0,1,3,2) #[50, 1, 3,288]
extra_tensor = extra_tensor.expand(50, 1279, 3, 288)  # [50, 1279, 3, 288]

# Concatenate
#combined_tensor = torch.cat([main_tensor, extra_tensor], dim=2)
# Step 2: Check if the dimensions match and then combine the tensors
if 'extra_tensor' in locals() or 'extra_tensor' in globals():
    print(f"extra_tensor shape: {extra_tensor.shape}")

    # Concatenate the tensors along the variable dimension (dimension 2)

    # Check if the first two dimensions (cases and sections) and the last dimension (time) match
    if batch_50.shape[:2] == extra_tensor.shape[:2] and batch_50.shape[3] == extra_tensor.shape[3]:
        combined_tensor = torch.cat([batch_50, extra_tensor], dim=2)
        print(f"Combined tensor shape: {combined_tensor.shape}")

        # Step 3: Save the tensor as a .pt file
        try:
            # Update variable names
            new_variable_names = variable_names + ['TT', 'PI', 'FI']
            print(f"Updated variable names: {new_variable_names}")
            saved_data["variable_names"] = new_variable_names
            saved_data["batch_50"] = combined_tensor

            torch.save(saved_data, combined_tensor_path)
            print(f"Combined tensor saved successfully to: {combined_tensor_path}")
        except Exception as e:
            print(f"Error saving the combined tensor: {e}")
            # Stop execution on save error
            raise

    else:
        print("Error: Tensor shapes are not compatible for concatenation along dimension 2.")
        print(f"full_tensor shape: {full_tensor.shape}")
        print(f"extra_tensor shape: {extra_tensor.shape}")
        # Stop execution on shape mismatch
        raise ValueError("Tensor shape mismatch for concatenation.")

else:
    print("Error: extra_tensor is not defined. Please ensure the previous cell was run.")
    # Stop execution if extra_tensor is not found
    raise NameError("extra_tensor is not defined.")

extra_tensor shape: torch.Size([50, 1279, 3, 288])
Combined tensor shape: torch.Size([50, 1279, 10, 288])
Updated variable names: ['PT', 'TM', 'HOL', 'HOLWT', 'UG', 'UL', 'ULWT', 'TT', 'PI', 'FI']
Combined tensor saved successfully to: /content/gdrive/MyDrive/Colab Notebooks/Transient Folder/FNO_dataset.pt


In [ ]:
import torch
import os

# Assuming dir_path is defined from the initial setup cells
# Define the path for the full tensor file and the combined tensor file
try:
    DIR = dir_path # Ensure DIR is set from dir_path
except NameError:
    print("Error: dir_path is not defined. Please run the initial setup cells.")
    raise # Stop execution if dir_path is not available

def P(fn: str) -> str:
    return os.path.join(DIR, fn)

FINAL_PATH = P("transient_model_full_dataset.pt")
combined_tensor_path = P("FNO_dataset.pt")


# Step 1: Load the transient_model_full_dataset.pt
try:
    saved_data = torch.load(FINAL_PATH)
    full_tensor = saved_data["full_tensor"]
    print(f"Loaded full_tensor with shape: {full_tensor.shape}")
    # Also get variable names if available
    variable_names = saved_data.get("variable_names", [])

except FileNotFoundError:
    print(f"Error: full_tensor file not found at {FINAL_PATH}")
    # Stop execution if the file is not found
    raise
except KeyError:
    print("Error: 'full_tensor' key not found in the loaded file.")
    # Stop execution if the key is not found
    raise
except Exception as e:
    print(f"An error occurred while loading full_tensor: {e}")
    # Stop execution for other loading errors
    raise


# Assuming extra_tensor is available from the previous cell (7Wo2RjlqhD9d and 4xlEkuD0oXgN)
# Step 2: Check if the dimensions match and then combine the tensors
if 'extra_tensor' in locals() or 'extra_tensor' in globals():
    print(f"extra_tensor shape: {extra_tensor.shape}")

    # Concatenate the tensors along the variable dimension (dimension 2)
    # Check if the first two dimensions (cases and sections) and the last dimension (time) match
    if batch_50.shape[:2] == extra_tensor.shape[:2] and batch_50.shape[3] == extra_tensor.shape[3]:
        combined_tensor = torch.cat([batch_50, extra_tensor], dim=2)
        print(f"Combined tensor shape: {combined_tensor.shape}")

        # Step 3: Save the tensor as a .pt file
        try:
            # Update variable names
            new_variable_names = variable_names + ['TT', 'PI', 'FI']
            print(f"Updated variable names: {new_variable_names}")
            saved_data["variable_names"] = new_variable_names
            saved_data["full_tensor"] = combined_tensor

            torch.save(saved_data, combined_tensor_path)
            print(f"Combined tensor saved successfully to: {combined_tensor_path}")
        except Exception as e:
            print(f"Error saving the combined tensor: {e}")
            # Stop execution on save error
            raise

    else:
        print("Error: Tensor shapes are not compatible for concatenation along dimension 2.")
        print(f"full_tensor shape: {batch_50.shape}")
        print(f"extra_tensor shape: {extra_tensor.shape}")
        # Stop execution on shape mismatch
        raise ValueError("Tensor shape mismatch for concatenation.")

else:
    print("Error: extra_tensor is not defined. Please ensure the previous cell was run.")
    # Stop execution if extra_tensor is not found
    raise NameError("extra_tensor is not defined.")

Loaded full_tensor with shape: torch.Size([150, 1279, 7, 288])
extra_tensor shape: torch.Size([50, 1279, 3, 288])
Combined tensor shape: torch.Size([50, 1279, 10, 288])
Updated variable names: ['PT', 'TM', 'HOL', 'HOLWT', 'UG', 'UL', 'ULWT', 'TT', 'PI', 'FI']
Combined tensor saved successfully to: /content/gdrive/MyDrive/Colab Notebooks/Transient Folder/FNO_dataset.pt
